# Revisiting the Cryptographic Hardness of Finding a Nash Equilibrium

### Sanjam Garg, Omkant Pandey, Askhayaram Srinivasan

##### Ashvin Jagadeesan, Akshat Yaparla

## The Question

BPR'15 prove the hardness of SVL by assuming the existence of quasi-polynomially hard iO, and sub-exponentially hard one-way functions. Can we base the hardness on SVL on commly assumed, polynomially hard, computational assumptions?


## The Answer, at a high level

Garg et al, proved that this question has an affirmative answer: it is possible to base SVL-hardness on simpler primitives. 

Namely, they showed that the existence of polynomially-hard iO and OWP implies SVL-hardness. 

## Main Contributions

Specifically, (some of) the major contributions of GPS'16 are based on:
- Relying on "regular strength"(polynomially hard) iO to construct a polynomial time-reduction.
    - This is in contrast to the reduction in BPR'15, which requires iO security with at least "super strong" (quasi-polynomially hard) security. 
- Reducing the number of hybrids in proof of hardness **without** loss in security
    - Specifically, the security of BPR'15 is based on either an exponential number of assumptions that arise from each pair of circuits or a polynomial set of assumptions with exponential loss in reduction.
    

## A Central Theorem

The entirety of this talk is based on the following theorem

**Theorem 1: OWP + iO $\rightarrow$ SVL-Hardness on Average**:

*That is, if polynomially-hard one-way permutations and indistinguishability obfuscation exist for P/poly, then SVL is hard, on average, for polynomial-time algorithms.*

## Goal of techniques

GPS'16 builds upon the work of BPR'15. Crucially, both papers show the construction of an efficient sampling procedure that samples hard instances of SVL. Today, we will see how to construct this sampler using polynomially-hard iO.

## Prelude: Some important notation and assumptions

In this presentation, we will utilize the following notation:
- Instances of SVL will be specified by a tuple $(S, V, x_s, T)$. 
    - $V$ is the verification circuit, where $V(x, i) = 1 \leftrightarrow x = S^{i - 1}(x_s)$
    - $T$ represents the sink that we wish to find, but will (supposedly) fail to do so efficiently
- Indices of $SVL$ instances will be represented by $i \in \{0, 1\}^{\kappa}$, and $T = 2^{\kappa} - 1$
- $i$ will be written in binary, and $i_{1:k}$ will represent the prefix string containing $i_1, i_2, ..., i_k$
- When integer $i$ is written in binary, it is written from left to right
- We have access to different $PRF$s, each of which can handle inputs of varying-length: 
    - so it is possible to have scenarios such as $PRF_{S_1}(0), PRF_{S_\kappa}(0^\kappa)$
- If $S$ appears as a subscript for a cryptographic primititive, it indicates a secret-key.
- In this presentation, the word "signature" corresponds to a value $\sigma$ that is an output of a PRF on some input

## BPR'15 Construction

The crux of the BPR'15 approach can be summarized as follows:

$KeyGen$: On input $1^\kappa$: Outputs $S_\kappa$ // **this is a secret key** not a circuit.

$O$: On input $S$: 
  - Outputs an indistinguishable obfuscation $\tilde{S}$

$S_{S_\kappa}$: On input $(i, \sigma)$:
- Checks if $\sigma = PRF_{S_\kappa}(i)$: 
     - If $i = T$:
          - Output $(i, PRF_{S_\kappa}(i))$
     - Else:
          - Output, $(i + 1, PRF_{S_\kappa}(i + 1))$.
- Else:
    - Output $\perp$

$V_{S_\kappa}$: On input $(x = (i, \sigma), j)$:
- If $i = j$:
    - Check if $\sigma = PRF_{S_\kappa}(i)$: // Equivalent to ($x = S^{i - 1}(x_s)$):
        - Output $1$
- Else:
    - Output $0$

$x_s = (1, PRF_{S_\kappa}(1))$
 
$T = 2^\kappa - 1$

Return instance $(O(S_{S_k}), O(V_{S_k}), (1, PRF_{S_{\kappa}}(1)), T)$


## An important Issue with BPR'15

Due to the fact that $T$ is a super-polynomial value, the size of the interval from $[u, T]$ may not be polynomial. As a result, iteratively puncturing the circuit leads to a super-polynomial number of hybrids in their proof of average-case hardness.


To address this issue, GPS'16 increases the punctured inveral whenever a node is made to be invalid

## Key Observations to improve BPR'15

For illustration, let us consider some index $u \in \{0, 1\}^{\kappa}$ such that it has $k$ trailing $1$s, where $k \in [\kappa]$. Then, $u$ will be of the following form 
$$u = u_{1: \kappa - k - 1} \cdot 01^k$$

Furthermore, $u + 1$ will be of the following form:
$$u + 1 = u_{1: \kappa - k - 1} \cdot 10^k$$

Notice that something interesting happens here due to the nature of prefix codes of binary representations. Namely,
that:
- The first $\kappa - k$ prefix bits of $u + 1$ are identical to the first $\kappa - k$ prefix bits of **all points** in the interval $[u + 1, u + 2^k]$
- The value $PRF_{S}((u + 1)_{1: \kappa - k})$ for index $u + 1$ is not needed  anywhere on the line graph except for $[u + 1, u + 2^k]$

## Exploiting the structure of prefix-codes

To exploit this fact, we introduce a slight modification to GPS'16. Rather than having each node be of the form:

$$(i, \sigma) \text{ where } \sigma = PRF_{S_\kappa}(i)$$

We have nodes of the following form:

$$(i, \sigma_{1}, \sigma_{2}, ..., \sigma_{\kappa})$$

Where $\sigma_j = PRF_{S_j}(i_{1:j})$ for all $j \in [\kappa]$, and each $S_j$ denotes a different secret key.

Note that $(i, \sigma_\kappa)$ corresponds to BPR'15

## GPS'16 Construction

$KeyGen$: On input $1^\kappa$:
- Output $\{S_k\}_{k \in [\kappa]}$, where each $S_k$ is a secret key for a $PRF_{S_k}:\{0, 1\}^k \rightarrow \{0, 1\}^\kappa$.

$O$: On input a circuit $S$:
- Outputs an indistinguishable obsfuscation $\tilde{S}$ of S (for circuits in P/poly)

$S_{S_1, S_2, ..., S_\kappa}$: On input $(i, \sigma_1, ..., \sigma_\kappa)$:
- If $\land_{k \in [\kappa]} \sigma_k = PRF_k(i_{1:k})$
    - If $i = T$:
        - Output $(i, \sigma_1, ..., \sigma_\kappa)$
    - Else:
        - Output $(i + 1, PRF_{S_1}((i + 1)_{1}), PRF_{S_2}((i + 1)_{1:2}), ..., PRF_{S_\kappa}((i + 1)_{1: \kappa})$
- Else:
    - Output $\perp$
    
$V_{S_1, S_2, ..., S_\kappa}$: On input $(x = (i, \sigma_1, ..., \sigma_\kappa), j)$:
- If $i = j$:
    - If $\land_{k \in \kappa}[\sigma_k = PRF_{S_k}(i_{1:k})]$:
        - Output $1$
- Output $0$

$x_s = (0^k, PRF_{S_1}(0), ..., PRF_{S_\kappa}(0^\kappa)$

$T = 2^\kappa - 1$

Return instance $(O(S_{S_1, S_2, ..., S_\kappa}), O(V_{S_1, S_2, ..., S_\kappa}), x_s, T)$

## From Lines to Chains

By constructing nodes in this manner, the signatures can be thought of as "virtual chains", emanating from each node and connecting to other nodes. 
- The first chain coming out of a node $i$ is connected to $i + 1$
- The second chain is connected to $i + 2$
- The $j^{th}$ chain is connected to $i + 2^{j - 1}$

The number of chains coming out of $i$ is one more than the number of trailing ones in the binary representation of $i$, or the number of bits that change from $i$ to $i + 1$.

## The Effects of puncturing

The successor circuit is now an obfuscation of a program that checks each signature on appropriate prefixes of $i$, and if so, signs all $\kappa$ prefixes of $i + 1$ using the appropriate keys. The first node is now:
$$x_s = (0^k, PRF_{S_1}(0), PRF_{S_2}(00), ..., PRF_{S_{\kappa}}(0^\kappa))$$

<!-- And $T = 2^{\kappa} - 1$ -->

Now, suppose that, as before, we have punctured the successor circuit at a random node $u$. Then the fact that the punctured circuit does not output *any* signature on $u + 1$ means that it does not output the signature $\sigma_{\kappa - k}$ on the first $\kappa - k$ bits of $u + 1$. 

The most important consequence of this result is that it does not output this signature on the first $\kappa - k$ bits of **any point in the interval** $[u + 1, u + 2^k]$. Puncturing with this type of node allows us to skip all the way to $[u + 2^k]$ are our next puncture point instead of $[u + 1]$. **This is one major improvement over BPR'15**, because we can now jump by a, potentially super-polynomial amount, to our next puncture point. We iterate over this process until we hit $T$. 

Puncturing the circuit is viewed as cutting the chains of appropriate length between points. While BPR cuts chains of length $1$, this strategy cuts the longest possible chain and then iterates. A consequence of this type of jump is that we will have a *polynomial number of hybrids* to consider.

## A Small Caveat

When implementing the idea above, we face the difficulty that for a random $u$, the number of chains coming out of $u$ could be very small(at least $1$). To circumvent this issue, we initially cut "smaller" length chains until we have the ability to cut "larger" length chains. This results in not needing to cut any more than a linear(in $\kappa$) number of chains in $T$. As a result, **the construction only suffers polynomial loss as opposed to sub-exponential loss in BPR'15** in the security parameter.

**Don't puncture and walk, puncture and jump**

## GPS'16 Construction

$KeyGen$: On input $1^\kappa$:
- Output $\{S_k\}_{k \in [\kappa]}$, where each $S_k$ is a secret key for a $PRF_{S_k}:\{0, 1\}^k \rightarrow \{0, 1\}^\kappa$.

$O$: On input a circuit $S$:
- Outputs an indistinguishable obsfuscation $\tilde{S}$ of S (for circuits in P/poly)

$S_{S_1, S_2, ..., S_\kappa}$: On input $(i, \sigma_1, ..., \sigma_\kappa)$:
- If $\land_{k \in [\kappa]} \sigma_k = PRF_k(i_{1:k})$
    - Output $(i + 1, PRF_{S_1}((i + 1)_{1}), PRF_{S_2}((i + 1)_{1:2}), ..., PRF_{S_\kappa}((i + 1)_{1: \kappa})$
- Else:
    - Output $\perp$
    
$V_{S_1, S_2, ..., S_\kappa}$: On input $(x = (i, \sigma_1, ..., \sigma_\kappa), j)$:
- If $i = j$:
    - If $\land_{k \in \kappa}[\sigma_k = PRF_{S_k}(i_{1:k})]$:
        - Output $1$
- Output $0$

$x_s = (0^k, PRF_{S_1}(0), ..., PRF_{S_\kappa}(0^\kappa)$

$T = 2^\kappa - 1$

Return instance $(O(S_{S_1, S_2, ..., S_\kappa}), O(V_{S_1, S_2, ..., S_\kappa}), x_s, T)$

## BPR'15 Construction

The crux of the BPR'15 approach can be summarized as follows:

$KeyGen$: On input $1^\kappa$: Outputs $S_\kappa$ // **this is a secret key** not a circuit.

$O$: On input $S$: 
  - Outputs an indistinguishable obfuscation $\tilde{S}$

$S_{S_\kappa}$: On input $(i, \sigma)$:
- Checks if $\sigma = PRF_{S_\kappa}(i)$:
    - Output, which is guaranteed to be $(i + 1, PRF_{S_\kappa}(i + 1))$.
- Else:
    - Output $\perp$

$V_{S_\kappa}$: On input $(x = (i, \sigma), j)$:
- If $i = j$:
    - Check if $\sigma = PRF_{S_\kappa}(i)$: // Equivalent to ($x = S^{i - 1}(x_s)$):
        - Output $1$
- Else:
    - Output $0$

$x_s = (1, PRF_{S_\kappa}(1))$
 
$T = 2^\kappa - 1$

Return instance $(O(S_{S_k}), O(V_{S_k}), (1, PRF_{S_{\kappa}}(1)), T)$



## Thanks!

*Appendix beyond this point*
*we will adhere with the notation of GPS'16, rather than BPR'15, where we cover proof of hardness**

## The SVL Sampler

Given the NEXT circuit, we can construct hard instances of SVL. Here is the recipe for doing so

$SAMPLER$: On input $1^\kappa$:
- Secret Keys: sample secret keys for PRF's, $\{S_i\}_{i \in [\kappa]} \leftarrow KeyGen_{PRF}(1^\kappa)$. For all $i \in [\kappa]$, $S_i$ is a seed for a $PRF_{S_i}:\{0, 1\}^i \rightarrow \{0, 1\}^\kappa$.
- Source Node: $x_s = (0^k, PRF_{S_1}(0), ..., PRF_{S_\kappa}(0^\kappa))$
- Successor Circuit: S \leftarrow iO(NEXT_{S_1, ..., S_\kappa})
- Verification Circuit: V where 

$V$: On input $((x, \sigma_1, ..., \sigma_\kappa), j)$
- If $x = j - 1$ and $iO(NEXT_{S_1, ..., S_\kappa})((x, \sigma_1, ..., \sigma_\kappa)) \neq \perp$: output 1
- Else: output 0
    


## Proof of Hardness: Strategy

We start by showing that $SAMPLER$ generates hard SVL instances. At a high level, it will suffice to show that:
- $V((x, \sigma_1, ..., \sigma_\kappa), j)$ outputs $1$ if and only if $x = j - 1$ and $NEXT_{S_1, ..., S_\kappa}(x, \sigma_1, ..., \sigma_\kappa) \neq \perp$
- $(x, \sigma_1, ..., \sigma_\kappa) = S^{i - 1}(x_s)$

(Therefore, by design, we have that $(x, \sigma_1, ..., \sigma_\kappa) = S^{i - 1}(x_s)$)
- No poly-time adversary can output a valid value for $(1^\kappa, \sigma_1, ..., \sigma_\kappa)$

## The Circuit $NEXT^*$

In our hybrids we will use the circuit $NEXT^*_{S_1, ..., S_\kappa, u, u'}$ which is a modification of $NEXT^{S_1, ..., S_{\kappa}}$ that is padded to size $p(\kappa)$. This circuit is identical to $NEXT^{S_1, ..., S_{\kappa}}$, except when $x \in [u, u']$, $NEXT^*_{S_1, .., S_\kappa, u, u'}$ outputs $\perp$.

## Sequence of Hybrids

Here are our Hybrids

## Hybrid 0

$Hyb_0$: This hybrid corresponds to the SVL instance generated by $SAMPLER(1^\kappa)$


## Hybrid 1

$Hyb_1$: In this hybrid, we change how the successor circuit, $S$, is generated. In particular:
- The new successor circuit $S \rightarrow iO(NEXT^1_{S_1, ..., S_k, v})$ instead of $iO(NEXT_{S_1, ..., S_\kappa})$, except on input $(x, \cdot, ..., \cdot)$, where $v = PRG(x)$, the circuit outputs $\perp$. The value $v \leftarrow_{U} \{0, 1\}^{2\kappa}$.
- As a result, $NEXT^1_{S_1, ..., S_\kappa, v}$ and $NEXT_{S_1, ..., S_\kappa}$ are functionally equivalent.
- Therefore iO implies computational indistinguishability between $Hyb_0$ and $Hyb_1$

## Hybrid 2

$Hyb_2$: In this hybrid, we change how the value $v$, that is hardcoded in $NEXT^1_{S_1, ..., S_\kappa}$, is generated. 
- Instead of sampling $v \leftarrow_U \{0, 1\}^{2\kappa}$, we generate $v = PRG(u_0)$ where $u \leftarrow_U \{0, 1\}^\kappa$. 
- Here, it is crucial that the PRG is length doubling and injective.
- The successor circuit is now $S \leftarrow iO(NEXT^1_{S_1, ..., S_\kappa, PRG(u_0)})$

## Hybrid 3

$Hyb_3$: In this hybrid, we change how the successor is generated. 
- Insteading setting the successor ciruit $S \leftarrow iO(NEXT^1_{S_1, ..., S_\kappa, PRG(u_0)})$, we set it to $S \leftarrow iO(NEXT^*_{S_1, ..., S_\kappa, u_0, u_0})$. 
- Due to the fact that the PRG is injective, iO implies that $Hyb_2$ and $Hyb_3$ are computationally indistinguishable.

## Prelude to Hybrid 4, j

Before we can proceed with $Hyb_{4, j}$, we need define values $u_j$. 

Let $u \in \{0, 1\}^\kappa$ be an index, let $f(u)$ denote the index of the lowest order bit of $u$ that is $0$(with index of highest order bit being $1$). Furthermore, let $\delta(u)$ is the number of 0 bits in $u$(when written in binary)

Formally,$f(u)$ is the smallest $j$ such that $u = u_{1:j} \cdot 1^{\kappa - j}$. 

For example, if $u = 10011$, then $f(u) = 3$

Starting with a value $u_0 \in \{0, 1\}^\kappa$, we define a sequence of values such that $u_{j + 1}$ is the value $u_j$ with $f(u_j)^{th}$ bit set to $1$. This implies that $u_{j + 1} = u_j + 2^{\kappa - f(u_j)}$

Now, we can see that for all $j \in \{0, ..., \delta(u_0)\}$, we have that $\delta(u_{j + 1}) < \delta(u_j)$ because we keep swapping values of $0$ with $1$ with each subsequent increment in $j$. 

Furthermore, we can note that $u_{\delta(u_0)} = 1^\kappa$ and $\delta(u_0) \leq \kappa$.

Let's walk through an example:
- Let $u_0 = 0010$, then $\kappa = 4$ and $\delta(u_0) = 3$. So now, the set in which $j$ can range is the set $\{0, ..., 3\}$
- Now, $f(u_0) = 4$
- Let's compute $u_1$. We take $u_0$ and set it's $f(u_0)^{th}$ bit to $1$. So, $u_1 = 0011$
- We can repeat this process to get $u_2 = 0111$ and $u_3 = 1111$

Notice how the size of the jump increases with each increment of $j$ increases. 

## Hybrid 4,j

$Hyb_{4, j}$: In this hybrid, for $j \in \{0, ..., \delta(u_0)\}$, the successor circuit is generated as an obfuscation of $NEXT^*_{S_1, ..., S_\kappa, u_0, u_j}$.

## Indistinguishability between Hybrid 4, j and Hybrid 4, j + 1

To prove this statement, we need to prove indistinguishability via sub-hybrids, where in each successive hybrid, we make a small change to the successor circuit. As a shorthand, let $f_j = f(u_j)$ and $t_j = u_{j_{1:f_j}} + 1$

### Hybrid 4, j, 1

$Hyb_{4, j, 1}$: In this hybrid, let $S'_{f_j} = Punc(S_{f_j}, t_j)$ and let $\sigma^* = PRF_{S_{f_j}}(t_j)$. Replace the successor circuit from an obfuscation of $NEXT^*_{S_1, ..., S_\kappa u_0, u_j}$ to an obfuscation of $NEXT^2_{S_1, ..., S_\kappa, u_0, u_j, \sigma^*}$

Note that $NEXT^2_{S_1, ..., S_\kappa, u_0, u_j, \sigma^*}$ is identical to $NEXT^*_{S_1, ..., S_\kappa u_0, u_j}$, except that it cannot compute $PRF_{S_{f_j}}(t_j)$. However, the same circuit is hardcoded with the value, denoted by $\sigma^*$. Now, $Hyb_{4, j}$ and $Hyb_{4, j, 1}$ are computationally indistinguishable

### Hybrid 4, j, 2

The successor circuit is still an obfuscation of $NEXT^2_{S_1, ..., S_\kappa, u_0, u_j, \sigma^*}$ with $S'_{f_j} = Punc(S_{f_j}, t_j)$(like in $Hyb_{4, j, 1})$. The difference between this hybrid and the previous is that $\sigma^* \leftarrow \{0, 1\}^\kappa$, rather than $PRF_{S_{f_j}}(t_j)$. Thus, $Hyb_{4, j, 1}$ and $Hyb_{4, j, 2}$ are computationally indistinguishable.

### Hybrid 4, j, 3

$Hyb_{4, j, 3}$: In this circuit, instead of generating the successor circuit as obfuscation of $NEXT^2_{S_1, ..., S_\kappa, u_0, u_j, \sigma^*}$, we generate it as an obfuscation of $NEXT^3_{S_1, ..., S_\kappa, u_0, u_j, \tau^*}$, where $\tau = PRG(\sigma^*)$.

This is essentially a way of getting around storing $\sigma^*$ in the clear. Note that $Hyb_{4, j, 2}$ and $Hyb_{4, j, 3}$ are computationally indistinguishable.

### Hybrid 4, j, 4

$Hyb_{4, j, 4}$: In this hybrid, we still obfuscate $NEXT^3_{S_1, ..., S_\kappa, u_0, u_j, \tau}$, except $\tau \leftarrow_U \{0, 1\}^{2\kappa}$ instead of $\tau = PRG(\sigma^*)$.

By the (indistinguishability )guarantees of PRG,$Hyb_{4, j, 3}$ and $Hyb_{4, j, 4}$ are computationally indistinguishable.

### Hybrid 4, j, 5

$Hyb_{4, j, 5}$: In this circuit, we take replace $NEXT^3_{S_1, ..., S_\kappa, u_0, u_j, \tau^*}$ with $iO(NEXT^3_{S_1, ..., S_\kappa, u_0, u_j, \tau^*})$.

Due to iO, $Hyb_{4, 1, 4}$ and $Hyb_{4, 1, 5}$ are computationally indistinguishable.

### Hybrid 4, j, 6

$Hyb_{4, j, 6}$: Instead of using the puncture key $S'_{f_j}$, we use the unpunctured key $S_{f_j}$. Specifically, we generate the sampler as an obfuscation of $NEXT^*{S_1, ..., S_k, u_0, u_{j + 1}}$

Finally, computational indistinguishability follows from iO 

Note that $Hyb_{4, j, 6}$ is the same as $Hyb_{4, j + 1}$. As a result, we have proven (computational) indistinguishability. Proving the hardness of the SVL instance

<!-- ## Analysis: From IO

These next sections will be concerned with the analysis behind this improvement. -->

<!-- ## Hard on Average SVL instances

Recall that we wish to prove the following

**Theorem 1: polynomially-hard OWP + iO for P/poly $\rightarrow$ PPAD-Hardness**:

That is, if polynomially-hard one-way permutations and indistinguishability obfuscation exist for P/poly, then EOL is hard for polynomial-time algorithms. -->

<!-- To begin, the sampler will generate a line graph. The nodes in the graph are of the form $(x, \sigma_1, ..., \sigma_{\kappa})$ where $x \in \{0, 1\}^\kappa$.

The nodes satisfy the following relation: for all $i \in [\kappa]$ if $PRF_{S_i}(x_{1:i}) = \sigma_{i}$, then $(x, \sigma_1, ..., \sigma_{\kappa})$ is valid. 

Furthermore, $(x, \sigma_1, ..., \sigma_{\kappa})$ is connected to $(x + 1, \sigma^{'}_1, ..., \sigma_{\kappa}^{'})$ via an out-going edge. Here $\sigma^{'}_{i} = PRF_{S_i}(x + 1_{1:i})$

Similarly, $(x, \sigma_1, ..., \sigma_{\kappa})$ is connected to $(x - 1, \sigma^{''}_1, ..., \sigma_{\kappa}^{''})$ symmetrically via an in-coming edge.

The source node is $(0, PRF_{S_1}(0), ..., PRF_{S_\kappa}(0^\kappa))$ -->

## Thanks!